In [1]:
import numpy as np

workDir = '/mnt/c/users/llave/Documents/CMS/'
data = np.load(workDir+'images_DYJets50V2.npy')
print(len(data))
print(data[0].shape)

8499
(40, 40, 3)


In [2]:
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.models import Model
from keras.optimizers import Adam
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import numpy as np

def build_discriminator(img_shape):
    input = Input(img_shape)
    x = Conv2D(32*3, kernel_size=(4,4), strides=(2,2), padding="same")(input)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = Conv2D(64*3, kernel_size=(4,4), strides=(2,2), padding="same")(x)
    x = ZeroPadding2D(padding=((0, 1), (0, 1)))(x)
    x = (LeakyReLU(alpha=0.2))(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Conv2D(128*3, kernel_size=(4,4), strides=(2,2), padding="same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Conv2D(256*3, kernel_size=(4,4), strides=(1,1), padding="same")(x)
    x = LeakyReLU(alpha=0.2)(x)
    x = Dropout(0.25)(x)
    x = Flatten()(x)
    #testing
    out = Dense(1, activation='sigmoid')(x)

    model = Model(input, out)
    print("-- Discriminator -- ")
    model.summary()
    return model


def build_generator(noise_shape=(100,)):
    input = Input(noise_shape)
    x = Dense(128 * 10 * 10, activation="relu")(input)
    x = Reshape((10,10, 128))(x)
    x = BatchNormalization(momentum=0.8)(x)
    #upsampling to 20x20
    x = Conv2DTranspose(128, (4,4), strides=(2,2), padding='same')(x)
    x = Activation("relu")(x)
    x = BatchNormalization(momentum=0.8)(x)
    #upsampling to 40x40
    x = Conv2DTranspose(64, (4,4),strides=(2,2), padding='same')(x)
    x = Activation("relu")(x)
    x = BatchNormalization(momentum=0.8)(x)
    x = Conv2D(3, kernel_size=3, padding="same")(x)
    out = Activation("tanh")(x)
    model = Model(input, out)
    print("-- Generator -- ")
    model.summary()
    return model

#plots events
def plot_event(eventNum):
    
    x = events[eventNum]
    
    fig, axs = plt.subplots(1,3)
    for i in range(3):
        axs[i].imshow(x[:,:,i])
        
    axs[0].set_title("ECAL")
    axs[1].set_title("HCAL")
    axs[2].set_title("Muon")
    
    plt.show()

#generates and saves 5 random images
def save_imgs(generator, epoch, batch):
    r, c = 5, 3
    noise = np.random.normal(0, 1, (r * c, 100))
    gen_imgs = generator.predict(noise)

    # Rescale images 0 - 1
    gen_imgs = 0.5 * gen_imgs + 0.5

    fig, axs = plt.subplots(r, c)
    cnt = 0
    for i in range(r):
        for j in range(c):
            axs[i, j].imshow(gen_imgs[cnt, :, :, j], cmap='gray')
            axs[i, j].axis('off')
            cnt += 1
    fig.savefig(workDir+"images/tracks_%d_%d.png" % (epoch, batch))
    plt.close()

Using TensorFlow backend.
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/llavezzo/.local/lib/python3.6/site-packages/tensorflo

In [3]:
#build and compile discriminator and generator
discriminator = build_discriminator(img_shape=(40,40, 3))
discriminator.compile(loss='binary_crossentropy',
                               optimizer=Adam(lr=0.0002, beta_1=0.5),
                               metrics=['mse'])

generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

-- Discriminator -- 
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 40, 40, 3)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 96)        4704      
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 20, 20, 96)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 20, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 10, 10, 192)       295104    
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 11, 11, 192)       0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 11

In [4]:
#combine them
z = Input(shape=(100,))
img = generator(z)
discriminator.trainable = False
real = discriminator(img)
combined = Model(z, real)
combined.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.0002, beta_1=0.5))

In [ ]:
import numpy.random
from numpy.random import choice
from numpy.random import randn

def noisy_labels(y, p_flip):
    # determine the number of labels to flip
    n_select = int(p_flip * y.shape[0])
    # choose labels to flip
    flip_ix = choice([i for i in range(y.shape[0])], size=n_select)
    # invert the labels in place
    y[flip_ix] = 1 - y[flip_ix]
    return y

# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
    # generate points in the latent space
    x_input = randn(latent_dim * n_samples)
    # reshape into a batch of inputs for the network
    x_input = x_input.reshape((n_samples, latent_dim))
    return x_input

def smooth_positive_labels(y):
    return y - 0.3 + (np.random.random(y.shape) * 0.5)

def smooth_negative_labels(y):
    return y + np.random.random(y.shape) * 0.3

X_train = data

epochs=100
batch_size=16
save_interval=1

num_examples = X_train.shape[0]
num_batches = int(num_examples / float(batch_size))
print('Number of examples: ', num_examples)
print('Number of Batches: ', num_batches)
print('Number of epochs: ', epochs)

half_batch = int(batch_size / 2)

d_loss = [10,10]
g_loss = 10
d_loss_array = []
g_loss_array = []

for epoch in range(epochs + 1):
    for batch in range(num_batches):

        # noise images for the batch
        noise = np.random.normal(0, 1, (half_batch, 100))
        fake_images = generator.predict(noise)
        fake_labels = np.zeros((half_batch, 1))

        # real images for batch
        idx = np.random.randint(0, X_train.shape[0], half_batch)
        real_images = X_train[idx]
        real_labels = np.ones((half_batch, 1))
        
        #testing noisy labels
        real_labels = noisy_labels(real_labels,0.05)
        real_labels = smooth_positive_labels(real_labels)
        fake_labels = smooth_negative_labels(fake_labels)

        # Train the discriminator (real classified as 1 and generated as 0)
        d_loss_real = discriminator.train_on_batch(real_images, real_labels)
        d_loss_fake = discriminator.train_on_batch(fake_images, fake_labels)
        d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

        # Train the generator
        labels = np.ones((batch_size, 1))

        noise = np.random.normal(0, 1, (batch_size, 100))
        #testing gaussian latent space
        noise = generate_latent_points(100,batch_size)
        
        g_loss = combined.train_on_batch(noise, labels)
            
        d_loss_array.append(d_loss[0])
        g_loss_array.append(g_loss)

        # Plot the progress
        if(batch % 50 == 0):
            #print("Epoch %d Batch %d/%d [D loss: %.2f, acc avg: %.2f%%] [D acc real: %.2f D acc fake: %.2f], [G loss: %.2f]" %
            #      (epoch, batch, num_batches, d_loss[0], 100 * d_loss[1], d_loss_real[1], d_loss_fake[1],g_loss))
             print("Epoch %d Batch %d/%d [D loss: %.2f, mse avg: %.2f] [D mse real: %.2f D mse fake: %.2f], [G loss: %.2f]" %
                  (epoch, batch, num_batches, d_loss[0], d_loss[1], d_loss_real[1], d_loss_fake[1],g_loss))
    
    
        if(batch % 50 == 0): save_imgs(generator, epoch, batch)


Number of examples:  8499
Number of Batches:  531
Number of epochs:  100
Epoch 0 Batch 0/531 [D loss: 0.24, mse avg: 0.02] [D mse real: 0.01 D mse fake: 0.03], [G loss: 2.85]
Epoch 0 Batch 50/531 [D loss: 0.22, mse avg: 0.01] [D mse real: 0.01 D mse fake: 0.00], [G loss: 4.87]
Epoch 0 Batch 100/531 [D loss: -0.24, mse avg: 0.02] [D mse real: 0.03 D mse fake: 0.00], [G loss: 5.81]
Epoch 0 Batch 150/531 [D loss: -0.22, mse avg: 0.01] [D mse real: 0.03 D mse fake: 0.00], [G loss: 7.07]
Epoch 0 Batch 200/531 [D loss: 0.05, mse avg: 0.01] [D mse real: 0.02 D mse fake: 0.00], [G loss: 7.87]
Epoch 0 Batch 250/531 [D loss: -0.05, mse avg: 0.01] [D mse real: 0.02 D mse fake: 0.00], [G loss: 6.36]
Epoch 0 Batch 300/531 [D loss: 0.07, mse avg: 0.00] [D mse real: 0.01 D mse fake: 0.00], [G loss: 6.86]
Epoch 0 Batch 350/531 [D loss: 0.59, mse avg: 0.12] [D mse real: 0.24 D mse fake: 0.00], [G loss: 56.69]
Epoch 0 Batch 400/531 [D loss: 0.34, mse avg: 0.06] [D mse real: 0.11 D mse fake: 0.01], [G lo

In [ ]:
plt.plot(d_loss_array,label='d_loss')
plt.plot(g_loss_array,label='g_loss')